In [33]:
import pandas as pd
import sys
sys.path.insert(0, '../')
import constants as const
import src.utils as util
import src.calculate_errors as err

In [5]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
lstm_iter_results = pd.read_csv('../lstm_new_results/rmse.csv')

In [4]:
lstm_iter_results.head()

,Unnamed: 0,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
0,32_3_0.001_1,0.461,0.455,0.447,0.342,0.342
1,32_3_0.001_3,0.440,0.448,0.491,0.357,0.355
2,32_3_0.001_7,0.465,0.446,0.472,0.373,0.363
3,32_3_0.0001_1,0.484,0.479,0.465,0.416,0.376
4,32_3_0.0001_3,0.451,0.441,0.454,0.452,0.485


In [19]:
fig = go.Figure()

for col in lstm_iter_results.columns[1:]:
    fig.add_trace(go.Scatter(x= lstm_iter_results['Unnamed: 0'], y=lstm_iter_results[col],  mode='markers', name=col))

In [20]:
fig.show()

In [18]:
lstm_iter_results.min()

Unnamed: 0           32_3_0.0001_1
Grid                          0.44
TL - Aggregated              0.441
SUB - Aggregated             0.441
PC - Aggregated              0.342
Site - Aggregated            0.332
dtype: object

In [78]:
cnn_iter_results = pd.read_csv('../cnn_new_results/rmse.csv')

In [79]:
cnn_iter_results.head()

,Unnamed: 0,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
0,32_4_0.001_1,0.546,0.465,0.467,0.323,0.332
1,32_4_0.001_3,0.583,0.471,0.486,0.498,0.573
2,32_4_0.001_7,0.564,0.572,0.503,0.769,0.778
3,32_4_0.0001_1,0.456,0.405,0.430,0.331,0.328
4,32_4_0.0001_3,0.446,0.442,0.455,0.357,0.376


In [80]:
cnn_iter_results.min()

Unnamed: 0           32_4_0.0001_1
Grid                         0.426
TL - Aggregated              0.405
SUB - Aggregated             0.394
PC - Aggregated              0.323
Site - Aggregated            0.325
dtype: object

In [81]:
fig = go.Figure()

for col in cnn_iter_results.columns[1:]:
    fig.add_trace(go.Scatter(x= cnn_iter_results['Unnamed: 0'], y=cnn_iter_results[col],  mode='markers', name=col))
    
fig.show()

## Grid + PC level NN

In [40]:
def sum_fc_results(ts_array, model_path):
    dfs = []
    for ts in ts_array:
        ts_fc = pd.read_csv(f'../{model_path}/{ts}.csv', index_col=[0])[['fc']]
        dfs.append(ts_fc)
    concat_df = pd.concat(dfs, axis=1).sum(axis=1)
    return concat_df

In [41]:
def calculate_grid_error(start, end, model_path, error_metric):
    if end == -1:
        ts_array = const.TS[start: ]
    else:
        ts_array = const.TS[start: end]
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = sum_fc_results(ts_array, model_path).values
    horizon = 14
    
    if error_metric == "RMSE":
        mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    else:
        mean_err, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
        
    return mean_err, error_dist

In [43]:
rmse_error, rmse_dist = calculate_grid_error(0, 1, 'combined_nn_results', "RMSE")

In [86]:
# rmse_dist
rmse_error

0.5857013669439975

In [55]:
rmse_error2, rmse_dist2 = calculate_grid_error(0, 1, 'combined_nn_results/loss_0.001', "RMSE")

In [56]:
rmse_error2

1.26047984138004

In [64]:
rmse_error3, rmse_dist3 = calculate_grid_error(0, 1, 'combined_nn_results/model1', "RMSE")

In [65]:
rmse_error3

0.5863197314945559

In [66]:
rmse_error4, rmse_dist4 = calculate_grid_error(0, 1, 'combined_nn_results/model2', "RMSE")

In [67]:
rmse_error4

0.6398894618933153

In [74]:
rmse_error5, rmse_dist5 = calculate_grid_error(0, 1, 'combined_nn_results/model3', "RMSE")

In [75]:
rmse_error5

1.0586457223920296

In [82]:
rmse_error6, rmse_dist6 = calculate_grid_error(0, 1, 'combined_nn_results/model4', "RMSE")

In [83]:
rmse_error6

0.7490994565629006

In [90]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(y = rmse_dist, name= "Current Approach (1)"))
fig_box.add_trace(go.Box(y = rmse_dist2, name= "Current Approach (2)"))
fig_box.add_trace(go.Box(y = rmse_dist3, name= "Current Approach (3)"))
fig_box.add_trace(go.Box(y = rmse_dist4, name= "Current Approach (4)"))
fig_box.add_trace(go.Box(y = rmse_dist5, name= "Current Approach (5)"))
fig_box.add_trace(go.Box(y = rmse_dist6, name= "Current Approach (6)"))
fig_box.show()

In [57]:
def read_loss_curve(path):
    return pd.read_pickle(f'../{path}/training_loss_grid_iteration')

In [58]:
def plot_loss(loss_array, models):

    fig = go.Figure()
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ))
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'))
        i +=1
    fig.show()

In [59]:
loss = read_loss_curve('combined_nn_results')

In [60]:
plot_loss([loss], ['combined_nn'])

In [76]:
loss = read_loss_curve('combined_nn_results/model3')

In [77]:
plot_loss([loss], ['combined_nn'])

plot forecasts

In [50]:
results_df = pd.read_csv(f'../combined_nn_results/grid.csv', index_col=[0])

In [51]:
results_df.head()

,power,fc
date_str,,
2021-01-09 07:00:00,11.51,94265.117188
2021-01-09 08:00:00,2827.61,94265.117188
2021-01-09 09:00:00,19125.99,94265.117188
2021-01-09 10:00:00,81339.20,94265.117188
2021-01-09 11:00:00,120248.84,145344.125000


In [54]:
fig_fc = go.Figure()

fig_fc.add_trace(go.Scatter(x=results_df.index, y = results_df['power'], name = 'actual'))
fig_fc.add_trace(go.Scatter(x=results_df.index, y = results_df['fc'], name = 'fc'))

fig_fc.show()